# Payroll Agent
- An agent which will interact with the user asking information
- based on the information give a decsion and execute a call by another agent which will send a mail regrading decsion by payroll agent

### Instatiating model

In [101]:
import warnings
warnings.filterwarnings('ignore')
from autogen import ConversableAgent


In [103]:
import os

config_list = {
        
        "model": "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        "api_key": os.environ.get("TOGETHERAI_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "stream": False,
    }

In [105]:
# Open the file in read mode
with open("policy.txt", "r") as file:
    content = file.read()  # Reads the entire content of the file
    print(content)


1. Name 
2. email id?
3. are you a trainer?
4. is your trip completed?
5.what is your ratings?
6.do you have manager approval?



In [107]:
payroll = ConversableAgent(
    name="payroll",
    system_message=f'''You are a helpful payroll team agent,
    you are here to help employees with travel and direct allowance.
    Your job is to gather employees information which is mandatory and following.
    {content}

    ask the complete information and make sure to ask the information one by one .
    bring all the information in the JSON format at the end
    

    if all the information is fullfilled .Fullfilled means he is a trainer,
    trip is completed and 
    rating is above 3 
    and manager approval is yes
    tell him that credit will be processed post 6:00PM today and 
    amount will be credited within 2-3 business days in registered bank account.
    
    if any of the condition is not fullfilled ,politely tell him the requirement to fullfill and 
    then again appply for allowances
    
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=config_list,
    code_execution_config=False,
    human_input_mode="NEVER",
)


#Non-LLM powered agent which will execute the tool
executive_mail =  ConversableAgent(
    name = 'executive_mail',
    llm_config=False, #this is a Non-LLM powered Agent
    
)




employee = ConversableAgent(
    name="employee",
    llm_config=False, #llm_config=False means that the ConversableAgent will not use a Language Model (LLM) to generate responses.
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

### Creating a tool

In [110]:
import smtplib

def send_mail(name:str,recipient:str,message:str)->str:

    subject = f'''Allowance Query'''
    message = f'''Dear {name},
                {message}'''
    
    my_email = "YOUR_MAIL_ID"
    password = "YOUR_APP_PASSWORD"

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as connection:
            connection.starttls()
            connection.login(user=my_email, password=password)
            email_message = f"Subject:{subject}\n\n{message}"
            connection.sendmail(from_addr=my_email, to_addrs=recipient, msg=email_message)
        return f"Email successfully sent to {recipient}!"
    except Exception as e:
        return f"Error sending email: {e}"

### Registering tool

In [113]:
#register tool
from autogen import register_function

# Register the book function to the two agents.
register_function(
    send_mail,
    caller=payroll,  # The assistant agent can suggest calls to the calculator.
    executor=executive_mail,  # The user proxy agent can execute the calculator calls.
    name="Mail_MAN",  # By default, the function name is used as the tool name.
    description="SEND Mail to employee after every decision",  # A description of the tool.
)


In [115]:
#create a task

chats = [
    {'sender':payroll,
     'recipient':employee,
     'message':'Hello Could you Please help me out with the information',
     #'summary_method':'last_msg',
     'summary_method':'reflection_with_llm',
     'max_turns':8,
     
    },
    {'sender':executive_mail,
     'recipient':payroll,
     'message':f'''Send the mail to the employee using the name and email id provided by the employee.
     Also use the decision at the end as the mail body the decision is the last message of payroll''',
     'max_turns':2,
     'summary_method':'reflection_with_llm'
    }
]

    

In [117]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
payroll (to employee):

Hello Could you Please help me out with the information

--------------------------------------------------------------------------------


Replying as employee. Provide feedback to payroll. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  ok


employee (to payroll):

ok

--------------------------------------------------------------------------------
payroll (to employee):

What is your name?

--------------------------------------------------------------------------------


Replying as employee. Provide feedback to payroll. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  shubhankar


employee (to payroll):

shubhankar

--------------------------------------------------------------------------------
payroll (to employee):

***** Suggested tool call (call_snj0owgmd013js0up63t3hlu): Mail_MAN *****
Arguments: 
{"name":"shubhankar","recipient":"shubhankar@example.com","message":"Hello, your information has been recorded."}
*************************************************************************

--------------------------------------------------------------------------------


...

employee (to payroll):

5

--------------------------------------------------------------------------------
payroll (to employee):

Do you have manager approval?

--------------------------------------------------------------------------------


Replying as employee. Provide feedback to payroll. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  yes


employee (to payroll):

yes

--------------------------------------------------------------------------------
payroll (to employee):

***** Suggested tool call (call_tljgvh8h5m2036awxrezd65f): Mail_MAN *****
Arguments: 
{"name":"shubhankar","recipient":"shubhankarsharma1395@gmail.com","message":"Credit will be processed post 6:00PM today and amount will be credited within 2-3 business days in registered bank account."}
*************************************************************************

--------------------------------------------------------------------------------


Replying as employee. Provide feedback to payroll. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  ok


employee (to payroll):

ok

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
executive_mail (to payroll):

Send the mail to the employee using the name and email id provided by the employee.
     Also use the decision at the end as the mail body the decision is the last message of payroll
Context: 
shubhankar is a trainer with a rating of 5, has manager approval, and their trip is completed, with an email sent to shubhankarsharma1395@gmail.com regarding credit processing.

--------------------------------------------------------------------------------
payroll (to executive_mail):

***** Suggested tool call (call_ddxbgh7cz5rbcgq0feqn9iqo): Mail_MAN *****
Arguments: 
{"name":"shubhankar","recipient":"shubhankarsharma1395@gmail.com","message":"Credit will be processed pos